In [ ]:
#Importing Libraries
import cv2
import os
import numpy as np
from keras.utils import np_utils
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation
from keras.optimizers import SGD
from keras.callbacks import ModelCheckpoint
from keras.models import load_model

In [ ]:
# Defining Variables and Setting path
images = []
targets = []

In [ ]:
# {with_mask : 0, without_mask : 1 (Viceversa)} - 2 Diffetent Outputs
prefix_dir_without_mask = 'dataset/without_mask/'
prefix_dir_with_mask = 'dataset/with_mask/'

In [ ]:
# Getting Image, Resizing, Converting It To Gray Scale, Storing in List Variables
content = os.listdir(prefix_dir_without_mask)
for image in content:
    try:
        image_path = prefix_dir_without_mask + image
        image = cv2.imread(image_path)
        image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        resized_image = cv2.resize(image_grey, (100, 100))
        images.append(resized_image)
        targets.append(1)
    except Exception as e:
        print("Exception occured as ",e)
    

In [ ]:
len(images)

In [ ]:
content = os.listdir(prefix_dir_with_mask)
for image in content:
    try:
        image_path = prefix_dir_with_mask + image
        image = cv2.imread(image_path)
        image_grey = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        resized_image = cv2.resize(image_grey, (100, 100))
        images.append(resized_image)
        targets.append(0)
    except Exception as e:
        print("Exception occured as ",e)

In [ ]:
len(images), len(targets)

In [ ]:
# Normalization
images = np.array(images)/255.0

In [ ]:
images.shape[0]

In [ ]:
images = np.resize(images, (3823, 100, 100, 1))

In [ ]:
# Converting To Binary Class Matrix
targets = np_utils.to_categorical(targets)

In [ ]:
# Defining list variable for training 
X_train, X_test, Y_train, Y_test = train_test_split(images, targets, test_size=0.25)

In [ ]:
# Creating Sequential model and Layers
model = Sequential()

model.add(Conv2D(200, (3,3), input_shape=(100, 100, 1)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))


model.add(Conv2D(100, (3,3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2,2)))

model.add(Dropout(0.4))
model.add(Flatten())

model.add(Dense(50, activation='relu'))
model.add(Dense(2, activation='softmax'))

# Compiling Model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics = ['accuracy'])

In [ ]:
# Saving Model For Future Use
cp = ModelCheckpoint('model-best', verbose=0, save_best_only=True)
# Training Model
model.fit(X_train, Y_train, epochs = 15,  callbacks=[cp], validation_split=0.2)

In [ ]:
# Loading Model
model = load_model('model-011.model')

In [ ]:
# For Face Detection
face_detect = cv2.CascadeClassifier('haarcascade_frontalface_default.xml')

In [ ]:
# Capturing Video
source = cv2.VideoCapture(0)

while 1:
    not_to_use, image = source.read()
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    faces = face_detect.detectMultiScale(gray, 1.5,5)
    for (x, y, w, h) in faces:
        face_roi = gray[y:y+w, x:x+w]
        resized_face = cv2.resize(face_roi, (100, 100))
        normalized_Face = resized_face/255
        reshaped_face = np.reshape(normalized_Face, (1, 100, 100, 1))
        result = model.predict(reshaped_face)[0]
        print("Probabilities : ",result)

        # Using 2 Outcomes as per definition - Mask_
        if result[0] > result[1]:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 255, 0), 2) 
            cv2.putText(image, "Safe", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
        

        if result[1] > result[0]:
            cv2.rectangle(image, (x, y), (x + w, y + h), (0, 0, 255), 2)
            cv2.putText(image, "Not Safe", (x, y), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 2)
            
    cv2.imshow('Verzeo Project', image)
    key = cv2.waitKey(1)
    if key == 27:
        break
        
cv2.destroyAllWindows()
source.release()
    